In [1]:
import pickle
import numpy as np
import sklearn
import scipy
from sklearn.metrics.pairwise import cosine_similarity as cos_sim

In [2]:
with open('/home/tomek/projects/paper-finder/app/pickles/full_cv.pkl', 'rb') as f:
    df_full_cv = pickle.load(f)
    
with open('/home/tomek/projects/paper-finder/app/pickles/vectorizer.pkl', 'rb') as f:
    tfidf_v = pickle.load(f)

/home/tomek/projects/paper-finder/venv/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/tomek/projects/paper-finder/venv/lib/python3.9/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [3]:
X_tfidf = tfidf_v.transform(df_full_cv.abstract)
X_tfidf_csc = X_tfidf.tocsc()

In [4]:
resnet_abstact = """
Deeper neural networks are more difficult to train. We present a residual learning framework to ease the training of networks that are substantially deeper than those used previously. We explicitly reformulate the layers as learning residual functions with reference to the layer inputs, instead of learning unreferenced functions. We provide comprehensive empirical evidence showing that these residual networks are easier to optimize, and can gain accuracy from considerably increased depth. On the ImageNet dataset we evaluate residual nets with a depth of up to 152 layers---8x deeper than VGG nets but still having lower complexity. An ensemble of these residual nets achieves 3.57% error on the ImageNet test set. This result won the 1st place on the ILSVRC 2015 classification task. We also present analysis on CIFAR-10 with 100 and 1000 layers.
The depth of representations is of central importance for many visual recognition tasks. Solely due to our extremely deep representations, we obtain a 28% relative improvement on the COCO object detection dataset. Deep residual nets are foundations of our submissions to ILSVRC & COCO 2015 competitions, where we also won the 1st places on the tasks of ImageNet detection, ImageNet localization, COCO detection, and COCO segmentation. 
"""

In [5]:
abstract = """Deeper neural networks"""

In [6]:
def calculate_cos_sim_old(abstract):
    
    x = tfidf_v.transform([abstract])
    sims = cos_sim(X_tfidf, x)
    sims = np.squeeze(sims)
    
    return sims

def calculate_cos_sim_fast(query_str: str):
    # Performs query only on columns existing in the 
    # Time complexity is O(n) where n is # of unique tokens
    # This is more efficinet, but only for small inputs 
    
    # tokenize the string
    x = np.array(tfidf_v.transform([query_str]).todense())
    
    # construct query vector
    x_nonzero_idx = x.nonzero()[1]
    x_small = x[:,x_nonzero_idx]
    
    # Narrow the similarity computation to tokens that are present in the query_string
    X_words_sparse = scipy.sparse.hstack([X_tfidf_csc.getcol(c) for c in x_nonzero_idx])

    # the vectors are already normalized
    # we can thus limit a*b/(norm(a)*norm(b)) to a*b
    cos_sim = X_words_sparse @ x_small.T
    
    return cos_sim

In [7]:
%timeit calculate_cos_sim_old(resnet_abstact)

59 ms ± 349 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%timeit calculate_cos_sim_fast(resnet_abstact)

10.6 ms ± 268 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%timeit calculate_cos_sim_fast(resnet_abstact[:5])

1.25 ms ± 24.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%timeit calculate_cos_sim_old(resnet_abstact[:10])

51.1 ms ± 413 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%timeit calculate_cos_sim_fast(resnet_abstact[:10])

1.15 ms ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
%timeit calculate_cos_sim_fast(resnet_abstact[:500])

5.11 ms ± 84.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
similarities = calculate_cos_sim_fast(resnet_abstact)

In [17]:
most_sim_idx = similarities.squeeze().argsort()[::-1]
df_similar = df_full_cv.iloc[most_sim_idx[:5]]
df_similar

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
686037,1512.03385,Kaiming He,"Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun",Deep Residual Learning for Image Recognition,Tech report,None,None,None,cs.CV,http://arxiv.org/licenses/nonexclusive-distrib...,Deeper neural networks are more difficult to...,"[{'version': 'v1', 'created': 'Thu, 10 Dec 201...",2015-12-11,"[[He, Kaiming, ], [Zhang, Xiangyu, ], [Ren, Sh..."
955911,1803.05778,Yatin Saraiya,Yatin Saraiya,Using accumulation to optimize deep residual n...,"7 pages, 6 figures, 1 table",None,None,None,cs.CV,http://arxiv.org/licenses/nonexclusive-distrib...,Residual Neural Networks [1] won first place...,"[{'version': 'v1', 'created': 'Sun, 14 Jan 201...",2018-03-16,"[[Saraiya, Yatin, ]]"
771150,1609.05672,Masoud Abdi,Masoud Abdi and Saeid Nahavandi,Multi-Residual Networks: Improving the Speed a...,This work has been submitted to the IEEE for p...,None,None,None,cs.CV,http://arxiv.org/licenses/nonexclusive-distrib...,"In this article, we take one step toward und...","[{'version': 'v1', 'created': 'Mon, 19 Sep 201...",2017-03-16,"[[Abdi, Masoud, ], [Nahavandi, Saeid, ]]"
758663,1608.02201,Hussein Al-Barazanchi,"Hussein A. Al-Barazanchi, Hussam Qassim, Abhis...",Residual CNDS,None,None,None,None,cs.CV,http://creativecommons.org/licenses/by/4.0/,Convolutional Neural networks nowadays are o...,"[{'version': 'v1', 'created': 'Sun, 7 Aug 2016...",2016-08-09,"[[Al-Barazanchi, Hussein A., ], [Qassim, Hussa..."
735261,1605.07146,Sergey Zagoruyko,"Sergey Zagoruyko, Nikos Komodakis",Wide Residual Networks,None,None,None,None,cs.CV cs.LG cs.NE,http://arxiv.org/licenses/nonexclusive-distrib...,Deep residual networks were shown to be able...,"[{'version': 'v1', 'created': 'Mon, 23 May 201...",2017-06-15,"[[Zagoruyko, Sergey, ], [Komodakis, Nikos, ]]"
